# 🗣️ **Speech To Text**


### 🛠 [Whisper by OpenAI](https://github.com/openai/whisper)


# Instructions for Using this Speech-to-Text Notebook

## Overview

This notebook provides a simple way to transcribe audio files to text using the Whisper model by OpenAI. It supports batch processing, various output formats, and can be run locally or in Google Colab.


**Key Features:**

* **Transcription:** Converts speech in audio files into written text.
* **Batch Processing:** Enables processing of multiple audio files or long audio files split into chunks.
* **Output Formats:** Supports various output formats like TXT, VTT, and SRT.

## Running in Google Colab

1. **Install Dependencies:** Run the provided code cell to install the required libraries.
2. **Upload Audio:** Upload your audio file(s) to the "Files" section in the Colab sidebar. For batch processing or long files, first split your audio into smaller chunks (around 10 minutes each) using the provided code cells.
3. **Configure Transcription:**
    * In the "Transcribe or Translate Recorded Audio" code cell, edit the `audio_file` path to match your uploaded file name or folder path.
    * Select your desired `use_model` (e.g., "tiny", "base", "medium").
    * Optionally, adjust other parameters like `language` and `prompt` as needed.
4. **Run Transcription:** Execute the "Transcribe or Translate Recorded Audio" code cell.
5. **Save Results:** Run the "Save results for recorded audio" code cell to save transcriptions in your chosen output formats (TXT, VTT, SRT) to the designated folder.
6. **Optional: Translate with DeepL:** If you have a DeepL API key, you can translate the transcriptions using the provided code cell.

## Running Locally

To use this code locally with an IDE (like VS Code, PyCharm), follow these steps:

1. **Create a Project:** Create a new project in your IDE.
2. **Install Libraries:** Install the necessary libraries listed in the "Step 1" section using `pip`:
3. **Download Code:** Copy the code from this Colab notebook and paste it into a Python file in your project (e.g., `audio_to_text.py`).
4. **Adapt Code:**
   * Update file paths for audio input and output to match your local setup.
   * Remove or comment out the Colab-specific parts of the code (e.g., `google.colab` imports, `eval_js` calls).
   * Consider streamlining the code by removing unused sections, such as the DeepL translation parts.  
5. **Execute:** Run your Python file (`audio_to_text.py`) from your IDE's terminal or by using the IDE's run functionality.

## Important Notes

* **Dependency Versions:** Please refer to the `requirements.txt` file for the specific dependency versions used during development. Different versions may lead to unexpected behavior.
* **Long Audio Files:** For optimal results with long audio files, use the provided code cells to split them into smaller chunks before transcription.
* **Prompting:** You can use the `prompt` parameter to provide context to the Whisper model and encourage a specific writing style.
* **DeepL Translation:** To use DeepL translation, you'll need a DeepL API key.


**Note**

This code was developed using specific dependency versions, and the Whisper model was not trained. Different versions or models may lead to varying results. Also the translation functionality using DeepL has been removed for this version.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.0
!pip install torch==2.6.0 torchvision==0.21.0+cu124
!pip install --no-cache-dir git+https://github.com/openai/whisper.git


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 50.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.1.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.7 

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2lnww7do
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2lnww7do
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


##⚙️ Install the required libraries

Click ▶️ button below to install the dependencies for this notebook.

In [ ]:
#@title { display-mode: "form" }
import subprocess

from sys import platform as sys_platform

status, ffmpeg_version = subprocess.getstatusoutput("ffmpeg -version")

if status != 0:
  from platform import platform

  if sys_platform == 'linux' and 'ubuntu' in platform().lower():
    !apt install ffmpeg
  else:
    print("Install ffmpeg: https://ffmpeg.org/download.html")
else:
  print(ffmpeg_version.split('\n')[0])

  NO_ROOT_WARNING = '|& grep -v \"WARNING: Running pip as the \'root\' user"' # running in Colab

  !pip install --no-warn-script-location --user --upgrade pip {NO_ROOT_WARNING}
  !pip install --root-user-action=ignore git+https://github.com/openai/whisper.git@v20231117 openai==1.9.0 numpy scipy deepl pydub cohere ffmpeg-python torch==2.1.0 tensorflow-probability==0.23.0 typing-extensions==4.9.0

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  Cloning https://github.com/openai/whisper.git (to revision v20231117) to /tmp/pip-req-build-u3dmgca_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-u3dmgca_
  Running command git checkout -q e58f28804528831904c3b6f2c0e473f346223433
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801443 sha256=e0bf582fcb310b3ec2e97be6f22dae6e1e0eff146a672b18f507e65de5b4a412
  Stored in directory: /tmp/pip-ephem-wheel-cache-huvfgaxp/wheels/af/ef/7e/5a10cc0c75969cb9d9b7f4ab7fe606bccabad41b56749760e4
Successfully built openai-whisper
  Attempting uninstall: openai-whisper
    Found exi

## 📁 Upload your audio files to the Files folder

⬅️ Files folder in Google Colab is on the left menu

Almost any audio or video file format is [supported](https://gist.github.com/Carleslc/1d6b922c8bf4a7e9627a6970d178b3a6).

## 🎙 Record your own audio ⏺

This is an **optional** step to record your microphone, useful if you do not have an audio file to upload and want to create one.

Run this cell to start recording your microphone.
A button will appear to stop the recording when you're done.

The recording will be saved as `recording.wav` which you can use in the next step `audio_file`.

In [ ]:
# This cell code is a slightly modified version of DotCSV code in the following Colab along with other references:
# https://colab.research.google.com/drive/1CvvYPAFemIZdSOt9fhN541esSlZR7Ic6?usp=sharing

try:
  import io
  import ffmpeg
  import numpy as np

  # Only available in Google Colab
  from google.colab.output import eval_js

  from IPython.display import HTML, Audio
  from scipy.io.wavfile import write, read as wav_read
  from base64 import b64decode
  from os.path import isfile

  AUDIO_HTML = """
  <script>
  var my_div = document.createElement("DIV");
  var my_p = document.createElement("P");
  var my_btn = document.createElement("BUTTON");
  var t = document.createTextNode("Starting recording...");

  my_btn.appendChild(t);
  my_div.appendChild(my_btn);
  document.body.appendChild(my_div);

  var base64data = 0;
  var reader;
  var recorder, gumStream;
  var recordButton = my_btn;

  var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
      bitsPerSecond: 16000,
      mimeType : 'audio/webm;codecs=opus' //codecs=pcm
    };
    recorder = new MediaRecorder(stream, options);
    //recorder = new MediaRecorder(stream);

    recorder.ondataavailable = function(e) {
      var url = URL.createObjectURL(e.data);
      var preview = document.createElement('audio');
      preview.controls = true;
      preview.src = url;
      document.body.appendChild(preview);

      reader = new FileReader();
      reader.readAsDataURL(e.data);
      reader.onloadend = function() {
        base64data = reader.result;
        //console.log("reader.onloadend: " + base64data);
      }
    };
    recorder.start();
    recordButton.innerText = "🔴 Recording... press to STOP";
  };

  navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

  function toggleRecording() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... please wait!";
    }
  }

  // https://stackoverflow.com/a/951057
  function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
  }

  var data = new Promise(resolve => {
    recordButton.onclick = () => {
      toggleRecording();

      sleep(2000).then(() => {
        // wait 2000ms for the data to be available...
        //console.log("resolve data: " + base64data);
        resolve(base64data.toString());
      });
    }
  });

  function doneRecording(recording_file) {
    my_div.removeChild(recordButton);
    my_p.innerText = recording_file;
    my_div.appendChild(my_p);
  }

  </script>
  """

  def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return audio, sr

  recording_file = "recording.wav" #@param {type:"string"}

  if isfile(recording_file):
    print(f"{recording_file} already exists, if you want to create another recording with the same name, delete it first")
  else:
    # record microphone
    audio, sr = get_audio()

    # write recording
    write(recording_file, sr, audio)

    eval_js(f'doneRecording("{recording_file}")')
except ImportError:
  print("Recording only available in Google Colab")

## 👂 Transcribe or Translate Recorded Audio

3.1. Choose a `task`:
  - `Transcribe` speech to text in the same language of the source audio file.
  - `Translate to English` speech to text in English.
  
Translation to other languages is not supported with _Whisper_ by default.
You may try to choose the _Transcribe_ task and set your desired `language`, but translation is not guaranteed. However, you can use **_DeepL_** later in the Step 5 to translate the transcription to another language.

3.2. Edit the `audio_file` to match your uploaded file name to transcribe.

- If you want to transcribe multiple files with the same parameters you must separate their file names with commas `,`

3.3. Run this cell and wait for the transcription to complete.

  You can try other parameters if the result with default parameters does not suit your needs.

  [Available models and languages](https://github.com/openai/whisper#available-models-and-languages)

  Setting the `language` to the language of source audio file may provide better results than Auto-Detect.

  You can add an optional initial `prompt` to provide context about the audio or encourage a specific writing style, see the [prompting guide](https://platform.openai.com/docs/guides/speech-to-text/prompting).

  If the execution takes too long to complete you can choose a smaller model in `use_model`, with an accuracy tradeoff, or use the OpenAI API.

  By default the open-source models are used, but you can also use the OpenAI API if the `api_key` parameter is set with your [OpenAI API Key](https://platform.openai.com/account/api-keys), which can improve the inference speed substantially, but it has an associated cost, see [API pricing](https://openai.com/pricing#audio-models).

  When using API some options are fixed: **use_model** is ignored (uses _large-v2_) and **coherence_preference** is ignored (uses _More coherence_).
  
  More parameters are available in the code `options` object.

In [ ]:
import os, subprocess

import whisper
from whisper.utils import format_timestamp, get_writer, WriteTXT

import numpy as np

try:
  import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
  pass

import torch

import math

from openai import OpenAI

# select task

task = "Transcribe" #@param ["Transcribe", "Translate to English"]

task = "transcribe" if task == "Transcribe" else "translate"

# select audio file

audio_file = "/content/drive/MyDrive/cmrg.mp3" #@param {type:"string"}

audio_files = list(map(lambda audio_path: audio_path.strip(), audio_file.split(',')))

for audio_path in audio_files:
  if not os.path.isfile(audio_path):
    raise FileNotFoundError(audio_path)

# set model

use_model = "medium" #@param ["tiny", "base", "small", "medium", "large-v1", "large-v2"]

# select language

language = "Auto-Detect" #@param ["Auto-Detect", "Afrikaans", "Albanian", "Amharic", "Arabic", "Armenian", "Assamese", "Azerbaijani", "Bashkir", "Basque", "Belarusian", "Bengali", "Bosnian", "Breton", "Bulgarian", "Burmese", "Castilian", "Catalan", "Chinese", "Croatian", "Czech", "Danish", "Dutch", "English", "Estonian", "Faroese", "Finnish", "Flemish", "French", "Galician", "Georgian", "German", "Greek", "Gujarati", "Haitian", "Haitian Creole", "Hausa", "Hawaiian", "Hebrew", "Hindi", "Hungarian", "Icelandic", "Indonesian", "Italian", "Japanese", "Javanese", "Kannada", "Kazakh", "Khmer", "Korean", "Lao", "Latin", "Latvian", "Letzeburgesch", "Lingala", "Lithuanian", "Luxembourgish", "Macedonian", "Malagasy", "Malay", "Malayalam", "Maltese", "Maori", "Marathi", "Moldavian", "Moldovan", "Mongolian", "Myanmar", "Nepali", "Norwegian", "Nynorsk", "Occitan", "Panjabi", "Pashto", "Persian", "Polish", "Portuguese", "Punjabi", "Pushto", "Romanian", "Russian", "Sanskrit", "Serbian", "Shona", "Sindhi", "Sinhala", "Sinhalese", "Slovak", "Slovenian", "Somali", "Spanish", "Sundanese", "Swahili", "Swedish", "Tagalog", "Tajik", "Tamil", "Tatar", "Telugu", "Thai", "Tibetan", "Turkish", "Turkmen", "Ukrainian", "Urdu", "Uzbek", "Valencian", "Vietnamese", "Welsh", "Yiddish", "Yoruba"]

# other parameters

prompt = "" #@param {type:"string"}

coherence_preference = "More coherence, but may repeat text" #@param ["More coherence, but may repeat text", "Less repetitions, but may have less coherence"]

api_key = '' #@param {type:"string"}

# detect device

if api_key:
  print("Using API")

  from pydub import AudioSegment
  from pydub.silence import split_on_silence
else:
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

  print(f"Using {'GPU' if DEVICE == 'cuda' else 'CPU ⚠️'}")

  # https://medium.com/analytics-vidhya/the-google-colab-system-specification-check-69d159597417
  if DEVICE == "cuda":
    !nvidia-smi -L
  else:
    if sys_platform == 'linux':
      !lscpu | grep "Model name" | awk '{$1=$1};1'

    print("Not using GPU can result in a very slow execution")
    print("Ensure Hardware accelerator by GPU is enabled in Google Colab: Runtime > Change runtime type")

    if use_model not in ['tiny', 'base', 'small']:
      print("You may also want to try a smaller model (tiny, base, small)")

# display language

WHISPER_LANGUAGES = [k.title() for k in whisper.tokenizer.TO_LANGUAGE_CODE.keys()]

if language == "Auto-Detect":
  language = "detect"

if language and language != "detect" and language not in WHISPER_LANGUAGES:
  print(f"\nLanguage '{language}' is invalid")
  language = "detect"

if language and language != "detect":
  print(f"\nLanguage: {language}")

# load model

if api_key:
  print()
else:
  MODELS_WITH_ENGLISH_VERSION = ["tiny", "base", "small", "medium"]

  if language == "English" and use_model in MODELS_WITH_ENGLISH_VERSION:
    use_model += ".en"

  print(f"\nLoading {use_model} model... {os.path.expanduser(f'~/.cache/whisper/{use_model}.pt')}")

  model = whisper.load_model(use_model, device=DEVICE)

  print(
      f"Model {use_model} is {'multilingual' if model.is_multilingual else 'English-only'} "
      f"and has {sum(np.prod(p.shape) for p in model.parameters()):,d} parameters.\n"
  )

# set options

## https://github.com/openai/whisper/blob/v20231117/whisper/transcribe.py#L37
## https://github.com/openai/whisper/blob/v20231117/whisper/decoding.py#L81
options = {
    'task': task,
    'verbose': True,
    'fp16': True,
    'best_of': 5,
    'beam_size': 5,
    'patience': None,
    'length_penalty': None,
    'suppress_tokens': '-1',
    'temperature': (0.0, 0.2, 0.4, 0.6, 0.8, 1.0), # float or tuple
    'condition_on_previous_text': coherence_preference == "More coherence, but may repeat text",
    'initial_prompt': prompt or None,
    'word_timestamps': False,
}

if api_key:
  api_client = OpenAI(api_key=api_key)

  api_supported_formats = ['mp3', 'mp4', 'mpeg', 'mpga', 'm4a', 'wav', 'webm']
  api_max_bytes = 25 * 1024 * 1024 # 25 MB

  api_transcribe = api_client.audio.transcriptions if task == 'transcribe' else api_client.audio.translations
  api_transcribe = api_transcribe.create

  api_model = 'whisper-1' # large-v2

  # https://platform.openai.com/docs/api-reference/audio?lang=python
  api_options = {
    'response_format': 'verbose_json',
  }

  if prompt:
    api_options['prompt'] = prompt

  api_temperature = options['temperature'][0] if isinstance(options['temperature'], (tuple, list)) else options['temperature']

  if isinstance(api_temperature, (float, int)):
    api_options['temperature'] = api_temperature
  else:
    raise ValueError("Invalid temperature type, it must be a float or a tuple of floats")
elif DEVICE == 'cpu':
  options['fp16'] = False
  torch.set_num_threads(os.cpu_count())

# execute task
# !whisper "{audio_file}" --task {task} --model {use_model} --output_dir {output_dir} --device {DEVICE} --verbose {options['verbose']}

if task == "translate":
  print("-- TRANSLATE TO ENGLISH --")
else:
  print("-- TRANSCRIPTION --")

results = {} # audio_path to result

for audio_path in audio_files:
  print(f"\nProcessing: {audio_path}\n")

  # detect language
  detect_language = not language or language == "detect"

  if not detect_language:
    options['language'] = language
    source_language_code = whisper.tokenizer.TO_LANGUAGE_CODE.get(language.lower())
  elif not api_key:
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio_path)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)

    source_language_code = max(probs, key=probs.get)
    options['language'] = whisper.tokenizer.LANGUAGES[source_language_code].title()

    print(f"Detected language: {options['language']}\n")

  # transcribe
  if api_key:
    # API
    if task == "transcribe" and not detect_language:
      api_options['language'] = source_language_code

    source_audio_name_path, source_audio_ext = os.path.splitext(audio_path)
    source_audio_ext = source_audio_ext[1:]

    if source_audio_ext in api_supported_formats:
      api_audio_path = audio_path
      api_audio_ext = source_audio_ext
    else:
      ## convert audio file to a supported format
      if options['verbose']:
        print(f"API supported formats: {','.join(api_supported_formats)}")
        print(f"Converting {source_audio_ext} audio to a supported format...")

      api_audio_ext = 'mp3'

      api_audio_path = f'{source_audio_name_path}.{api_audio_ext}'

      subprocess.run(['ffmpeg', '-i', audio_path, api_audio_path], check=True, capture_output=True)

      if options['verbose']:
        print(api_audio_path, end='\n\n')

    ## split audio file in chunks
    api_audio_chunks = []

    audio_bytes = os.path.getsize(api_audio_path)

    if audio_bytes >= api_max_bytes:
      if options['verbose']:
        print(f"Audio exceeds API maximum allowed file size.\nSplitting audio in chunks...")

      audio_segment_file = AudioSegment.from_file(api_audio_path, api_audio_ext)

      min_chunks = math.ceil(audio_bytes / (api_max_bytes / 2))

      # print(f"Min chunks: {min_chunks}")

      max_chunk_milliseconds = int(len(audio_segment_file) // min_chunks)

      # print(f"Max chunk milliseconds: {max_chunk_milliseconds}")

      def add_chunk(api_audio_chunk):
        api_audio_chunk_path = f"{source_audio_name_path}_{len(api_audio_chunks) + 1}.{api_audio_ext}"
        api_audio_chunk.export(api_audio_chunk_path, format=api_audio_ext)
        api_audio_chunks.append(api_audio_chunk_path)

      def raw_split(big_chunk):
        subchunks = math.ceil(len(big_chunk) / max_chunk_milliseconds)

        for subchunk_i in range(subchunks):
          chunk_start = max_chunk_milliseconds * subchunk_i
          chunk_end = min(max_chunk_milliseconds * (subchunk_i + 1), len(big_chunk))
          add_chunk(big_chunk[chunk_start:chunk_end])

      non_silent_chunks = split_on_silence(audio_segment_file,
                                           seek_step=5, # ms
                                           min_silence_len=1250, # ms
                                           silence_thresh=-25, # dB
                                           keep_silence=True) # needed to aggregate timestamps

      # print(f"Non silent chunks: {len(non_silent_chunks)}")

      current_chunk = non_silent_chunks[0] if non_silent_chunks else audio_segment_file

      for next_chunk in non_silent_chunks[1:]:
        if len(current_chunk) > max_chunk_milliseconds:
          raw_split(current_chunk)
          current_chunk = next_chunk
        elif len(current_chunk) + len(next_chunk) <= max_chunk_milliseconds:
          current_chunk += next_chunk
        else:
          add_chunk(current_chunk)
          current_chunk = next_chunk

      if len(current_chunk) > max_chunk_milliseconds:
        raw_split(current_chunk)
      else:
        add_chunk(current_chunk)

      if options['verbose']:
        print(f'Total chunks: {len(api_audio_chunks)}\n')
    else:
      api_audio_chunks.append(api_audio_path)

    ## process chunks
    result = None

    for api_audio_chunk_path in api_audio_chunks:
      ## API request
      with open(api_audio_chunk_path, 'rb') as api_audio_file:
        api_result = api_transcribe(model=api_model, file=api_audio_file, **api_options)
        api_result = api_result.model_dump() # to dict

      api_segments = api_result['segments']

      if result:
        ## update timestamps
        last_segment_timestamp = result['segments'][-1]['end'] if result['segments'] else 0

        for segment in api_segments:
          segment['start'] += last_segment_timestamp
          segment['end'] += last_segment_timestamp

        ## append new segments
        result['segments'].extend(api_segments)

        if 'duration' in result:
          result['duration'] += api_result.get('duration', 0)
      else:
        ## first request
        result = api_result

        if detect_language:
          print(f"Detected language: {result['language'].title()}\n")

      ## display segments
      if options['verbose']:
        for segment in api_segments:
          print(f"[{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}] {segment['text']}")
  else:
    # Open-Source
    result = whisper.transcribe(model, audio_path, **options)

  # fix results formatting
  for segment in result['segments']:
    segment['text'] = segment['text'].strip()

  result['text'] = '\n'.join(map(lambda segment: segment['text'], result['segments']))

  # set results for this audio file
  results[audio_path] = result

Using GPU
GPU 0: Tesla T4 (UUID: GPU-9f7ecd66-804f-2f0e-b4cb-6d1c9d5925c9)

Loading medium model... /root/.cache/whisper/medium.pt


100%|█████████████████████████████████████| 1.42G/1.42G [00:33<00:00, 45.0MiB/s]


RuntimeError: Numpy is not available

In [ ]:
import numpy as np
np.__version__


'2.0.2'

In [ ]:
!pip install --no-cache-dir --force-reinstall "numpy<2"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 242.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.1.0 which is incompatible.


In [ ]:
!pip install --upgrade numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 149.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.1.0 which is incompatible.


In [ ]:
!pip install --upgrade numpy
!pip install --upgrade --no-cache-dir git+https://github.com/openai/whisper.git

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.1.0 which is incompatible.
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-moouhuva
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-moouhuva

In [ ]:
import whisper


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [ ]:
#Importing Required libraries for Transcription

import os, subprocess

import whisper
from whisper.utils import format_timestamp, get_writer, WriteTXT

import numpy as np

try:
  import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
  pass

import torch

import math

from openai import OpenAI


## **Splitting Long Audio Files into Chunks**

This code cell is designed to handle long audio files by splitting them into smaller, manageable chunks. This is particularly useful for:

* **Improving Transcription Accuracy:** Whisper models perform best on shorter audio segments.
* **Reducing Memory Usage:** Processing large files directly can strain system resources.
* **Enabling Batch Processing:** Splitting files allows for parallel transcription of chunks.

**How it Works:**

1. **Input:** Specify the `audio_folder` containing your long audio files (MP3, WAV, M4A, or MP4).
2. **Output:** Specify the `output_base_folder` where the split chunks will be stored.
3. **Chunk Length:** The code splits audio into chunks of 10 minutes (600,000 milliseconds) by default (this is to improve clarity of transcribed text). You can adjust the `chunk_length` parameter if needed.
4. **Format Conversion:**  All audio chunks are converted to WAV format with a single channel (mono) and a sample rate of 16kHz, which is optimized for Whisper.
5. **File Naming:**  Chunks are named with the original file name plus a part number (e.g., `audio_part1.wav`, `audio_part2.wav`).
6. **Subfolders:** For organization, each input audio file's chunks are stored in a separate subfolder within the `output_base_folder`.

**Running the Code:**

1. **Configure Paths:** Update the `audio_folder` and `output_base_folder` variables to match your desired locations.
2. **Execute:** Run the code cell. The audio files will be split, converted, and saved into the specified output folder structure.

In [ ]:
import os
from pydub import AudioSegment

# Function to create a folder if it doesn't exist
def ensure_folder_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# Function to split and export audio as WAV
def split_audio_to_wav(input_file, output_folder, chunk_length=600000):  # 600000ms = 10 minutes
    try:
        ensure_folder_exists(output_folder)  # Ensure the subfolder exists

        # Load the audio file (auto-detect format)
        audio = AudioSegment.from_file(input_file)

        # Convert to mono and 16kHz (optimized for Whisper)
        audio = audio.set_channels(1).set_frame_rate(16000)

        # Get total length of audio
        total_length = len(audio)
        num_chunks = (total_length // chunk_length) + (1 if total_length % chunk_length else 0)

        # Split and save chunks
        for i in range(num_chunks):
            start_time = i * chunk_length
            end_time = min((i + 1) * chunk_length, total_length)

            chunk = audio[start_time:end_time]
            chunk_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(input_file))[0]}_part{i+1}.wav")
            chunk.export(chunk_filename, format="wav")

            print(f"Saved chunk: {chunk_filename}")

    except Exception as e:
        print(f"Error processing {input_file}: {e}")

# Define input and output folders
audio_folder = "/content/drive/MyDrive/Audio Type"  # Folder with full-length audio files
output_base_folder = "/content/drive/MyDrive/AudioSplit_cmrg"  # Base folder for split files

ensure_folder_exists(output_base_folder)  # Ensure the base output folder exists

# Iterate through each file in the input folder
for filename in os.listdir(audio_folder):
    if filename.endswith((".mp3", ".wav", ".m4a", "mp4")):  # ✅ Now supports M4A files
        input_path = os.path.join(audio_folder, filename)

        # Create a subfolder for each audio file inside the output base folder
        output_path = os.path.join(output_base_folder, os.path.splitext(filename)[0])

        ensure_folder_exists(output_path)  # Ensure subfolder exists
        split_audio_to_wav(input_path, output_path)


Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part1.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part2.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part3.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part4.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part5.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part6.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part7.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part8.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part9.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part10.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part11.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part12.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmrg_part13.wav
Saved chunk: /content/drive/MyDrive/AudioSplit_cmrg/cmrg/cmr

In [ ]:
# @title For Splitting Single(i.e. not in batches) Audio files
input_file = "" # @param {"type":"string","placeholder":"Audio File Path"}
output_folder = "" # @param {"type":"string","placeholder":"Output folder path "}
import os
from pydub import AudioSegment

# Function to create a folder if it doesn't exist
def ensure_folder_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# Function to split and export audio as WAV
def split_audio_to_wav(input_file, output_folder, chunk_length=600000):  # 600000ms = 10 minutes
    try:
        ensure_folder_exists(output_folder)  # Ensure the subfolder exists

        # Load the audio file (auto-detect format)
        audio = AudioSegment.from_file(input_file)

        # Convert to mono and 16kHz (optimized for Whisper)
        audio = audio.set_channels(1).set_frame_rate(16000)

        # Get total length of audio
        total_length = len(audio)
        num_chunks = (total_length // chunk_length) + (1 if total_length % chunk_length else 0)

        # Split and save chunks
        for i in range(num_chunks):
            start_time = i * chunk_length
            end_time = min((i + 1) * chunk_length, total_length)

            chunk = audio[start_time:end_time]
            chunk_filename = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(input_file))[0]}_part{i+1}.wav")
            chunk.export(chunk_filename, format="wav")

            print(f"Saved chunk: {chunk_filename}")

    except Exception as e:
        print(f"Error processing {input_file}: {e}")



# Split the single audio file
split_audio_to_wav(input_file, output_folder)

## **Batch Transcription and 💾 Save Text Combination**

This code cell performs the following steps:

1. **Iterate through Audio Chunks:** It iterates through the subfolders within the specified `split_audio_base_folder`, which should contain the 10-minute audio chunks generated by the audio splitting code.
2. **Transcribe Each Chunk:** For each audio file within a subfolder, it utilizes the Whisper model to transcribe the audio content.
3. **Combine Transcriptions:** It combines the transcriptions of all chunks belonging to the same original audio file into a single text file.
4. **Save Combined Transcription:** It saves the combined transcription to a new folder within your Google Drive, named `output_folder`.

**How it Works:**

* **Input:**
    * `split_audio_base_folder`: The path to the folder containing the subfolders with audio chunks.
    * `output_folder`: The path to the folder on your Google Drive where the combined transcriptions will be stored.
* **Whisper Model:** It loads the specified Whisper model (`use_model`) and uses it for transcription.
* **Language:** By default, it forces transcription in English using `language="en"`. You can modify this if needed.
* **Output:**  It creates a text file for each original audio file, containing the combined transcription of all its chunks. The file is named after the original audio file and stored in the `output_folder`.

**Running the Code:**

1. **Configure Paths:** Update the `split_audio_base_folder` and `output_folder` variables to match your file structure on Google Drive.
2. **Select Model:** Choose the appropriate Whisper model (`use_model`) based on your desired accuracy and speed trade-off.
3. **Execute:** Run the code cell. It will process all audio chunks, combine their transcriptions, and store the results in the specified output folder.

In [ ]:
import whisper
model = whisper.load_model("medium")

In [ ]:
import os
import torch
#import whisper
from whisper.utils import format_timestamp
#import numpy as np


# Define paths
split_audio_base_folder = "/content/drive/MyDrive/AudioSplit_cmrg"  # Folder containing subfolders with audio chunks
output_folder = "/content/drive/MyDrive/Transcriptions_cmrg"  # Folder to store transcriptions

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load Whisper model
use_model = "medium"  # Change model as needed
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# Iterate through subfolders
for subfolder in sorted(os.listdir(split_audio_base_folder)):
    subfolder_path = os.path.join(split_audio_base_folder, subfolder)
    if not os.path.isdir(subfolder_path):
        continue  # Skip non-folder files

    print(f"Processing folder: {subfolder}")
    all_text = []

    # Iterate through audio files in the subfolder
    for audio_file in sorted(os.listdir(subfolder_path)):
        if not audio_file.lower().endswith((".wav", ".mp3", ".mp4", "m4a")):
            continue  # Skip non-audio files

        audio_path = os.path.join(subfolder_path, audio_file)
        print(f"  Transcribing: {audio_file}")

        # Transcribe audio
        result = model.transcribe(audio_path, language="en")  # Force transcription in English

        # Store transcribed text
        for segment in result['segments']:
            all_text.append(f"[{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}] {segment['text']}")

    # Save transcription to output folder
    output_txt_path = os.path.join(output_folder, f"{subfolder}.txt")
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_text))

    print(f"Saved transcription: {output_txt_path}\n")

print("All transcriptions completed!")


Processing folder: .ipynb_checkpoints
Saved transcription: /content/drive/MyDrive/Transcriptions_cmrg/.ipynb_checkpoints.txt

Processing folder: cmrg
  Transcribing: cmrg_part1.wav
  Transcribing: cmrg_part10.wav
  Transcribing: cmrg_part11.wav
  Transcribing: cmrg_part12.wav
  Transcribing: cmrg_part13.wav
  Transcribing: cmrg_part14.wav
  Transcribing: cmrg_part15.wav
  Transcribing: cmrg_part2.wav
  Transcribing: cmrg_part3.wav
  Transcribing: cmrg_part4.wav
  Transcribing: cmrg_part5.wav
  Transcribing: cmrg_part6.wav
  Transcribing: cmrg_part7.wav
  Transcribing: cmrg_part8.wav
  Transcribing: cmrg_part9.wav
Saved transcription: /content/drive/MyDrive/Transcriptions_cmrg/cmrg.txt

All transcriptions completed!


## 💾 **Save results for recorded audio**

Run this cell to write the transcription as a file output.

Results will be available in the **audio_transcription** folder in the formats selected in `output_formats`.

If you don't see that folder, you may need to refresh 🔄 the Files folder.

Available formats: `txt,vtt,srt,tsv,json`

In [ ]:
# set output folder
output_dir = "audio_transcription"

# set output formats: https://github.com/openai/whisper/blob/v20231117/whisper/utils.py#L283
output_formats = "txt,vtt,srt" #@param ["txt,vtt,srt,tsv,json", "txt,vtt,srt", "txt,vtt", "txt,srt", "txt", "vtt", "srt", "tsv", "json"] {allow-input: true}
output_formats = output_formats.split(',')

from typing import TextIO

class WriteText(WriteTXT):

  def write_result(self, result: dict, file: TextIO, **kwargs):
    print(result['text'], file=file, flush=True)

def write_result(result, output_format, output_file_name):
  output_format = output_format.strip()

  # start captions in non-zero timestamp (some media players does not detect the first caption)
  fix_vtt = output_format == 'vtt' and result['segments'] and result['segments'][0].get('start') == 0

  if fix_vtt:
    result['segments'][0]['start'] += 1/1000 # +1ms

  # write result in the desired format
  writer = WriteText(output_dir) if output_format == 'txt' else get_writer(output_format, output_dir)
  writer(result, output_file_name)

  if fix_vtt:
    result['segments'][0]['start'] = 0 # reset change

  output_file_path = os.path.join(output_dir, f"{output_file_name}.{output_format}")
  print(output_file_path)

# save results

print("Writing results...")

os.makedirs(output_dir, exist_ok=True)

for audio_path, result in results.items():
  print(end='\n')

  output_file_name = os.path.splitext(os.path.basename(audio_path))[0]

  for output_format in output_formats:
    write_result(result, output_format, output_file_name)

##💬 Translate results with DeepL (API key needed)

This is an **optional** step to translate the transcription to another language using the **DeepL** API.

[Get a DeepL Developer Account API Key](https://www.deepl.com/pro-api)

Set the `deepl_api_key` to translate the transcription to a supported language in `deepl_target_language`.

In [ ]:
import deepl

# translation service options (DeepL Developer Account)

deepl_api_key = "" #@param {type:"string"}

deepl_target_language = "" #@param ["", "Bulgarian", "Chinese (simplified)", "Czech", "Danish", "Dutch", "English (American)", "English (British)", "Estonian", "Finnish", "French", "German", "Greek", "Hungarian", "Indonesian", "Italian", "Japanese", "Korean", "Latvian", "Lithuanian", "Norwegian", "Polish", "Portuguese (Brazilian)", "Portuguese (European)", "Romanian", "Russian", "Slovak", "Slovenian", "Spanish", "Swedish", "Turkish", "Ukrainian"]

deepl_formality = "default" #@param ["default", "formal", "informal"]

deepl_coherence_preference = "Share context between lines" #@param ["Share context between lines", "Translate each line independently"]
deepl_coherence_preference = deepl_coherence_preference == "Share context between lines"

if not deepl_api_key:
  print("Required: deepl_api_key")
  print("Get a DeepL Developer Account API Key: https://www.deepl.com/pro-api")

if not deepl_target_language:
  print("Required: deepl_target_language")
elif deepl_target_language == 'English':
  deepl_target_language = "English (British)"
elif deepl_target_language == 'Chinese':
  deepl_target_language = "Chinese (simplified)"
elif deepl_target_language == 'Portuguese':
  deepl_target_language = "Portuguese (European)"

use_deepl_translation = deepl_api_key and deepl_target_language

if use_deepl_translation:
  if deepl_formality != 'default':
    deepl_formality = 'prefer_more' if deepl_formality == 'formal' else 'prefer_less'

  translated_results = {} # audio_path to translated results

  try:
    deepl_translator = deepl.Translator(deepl_api_key)

    deepl_source_languages = [lang.code.upper() for lang in deepl_translator.get_source_languages()]

    deepl_target_languages_dict = deepl_translator.get_target_languages()
    deepl_target_languages = [lang.name for lang in deepl_target_languages_dict]

    deepl_target_language_code = next(lang.code for lang in deepl_target_languages_dict if lang.name == deepl_target_language).upper()
    target_language_code = deepl_target_language_code.split('-')[0]

    for audio_path, result in results.items():
      deepl_usage = deepl_translator.get_usage()

      if deepl_usage.any_limit_reached:
        print(audio_path)
        raise deepl.DeepLException("Quota for this billing period has been exceeded, message: Quota Exceeded")
      else:
        print(audio_path + '\n')

      # translate results (DeepL)
      source_language_code = whisper.tokenizer.TO_LANGUAGE_CODE.get(result['language'].lower()).upper()

      if (task == 'translate' and target_language_code != 'EN') or (task == 'transcribe' and source_language_code in deepl_source_languages and source_language_code != target_language_code):
        source_lang = source_language_code if task == 'transcribe' else None
        translate_from = f"from {result['language'].title()} [{source_language_code}] " if source_lang else ''
        print(f"DeepL: Translate results {translate_from}to {deepl_target_language} [{deepl_target_language_code}]\n")

        segments = result['segments']

        translated_results[audio_path] = { 'text': '', 'segments': [], 'language': deepl_target_language }

        # segments / request (max 128 KiB / request, so deepl_batch_requests_size is limited to around 1000)
        deepl_batch_requests_size = 200 # 200 segments * ~100 bytes / segment = ~20 KB / request  (~15 minutes of speech)

        for batch_segments in [segments[i:i + deepl_batch_requests_size] for i in range(0, len(segments), deepl_batch_requests_size)]:
          batch_segments_text = [segment['text'] for segment in batch_segments]

          if deepl_coherence_preference:
            batch_segments_text = '<br/>'.join(batch_segments_text)

          # DeepL request
          deepl_results = deepl_translator.translate_text(
              text=batch_segments_text,
              source_lang=source_lang,
              target_lang=deepl_target_language_code,
              formality=deepl_formality,
              split_sentences='nonewlines',
              tag_handling='xml' if deepl_coherence_preference else None,
              ignore_tags='br' if deepl_coherence_preference else None, # used to synchronize sentences with whisper lines but without splitting sentences in DeepL
              outline_detection=False if deepl_coherence_preference else None
          )

          deepl_results_segments = deepl_results.text.split('<br/>') if deepl_coherence_preference else [deepl_result_segment.text for deepl_result_segment in deepl_results]

          for j, translated_text in enumerate(deepl_results_segments):
            segment = batch_segments[j]

            # fix sentence formatting
            translated_text = translated_text.lstrip(',.。 ').rstrip()

            if not deepl_coherence_preference and translated_text and translated_text[-1] in '.。' and segment['text'][-1] not in '.。':
              translated_text = translated_text[:-1]

            # add translated segments
            translated_results[audio_path]['segments'].append(dict(id=segment['id'], start=segment['start'], end=segment['end'], text=translated_text))

            if options['verbose']:
              print(f"[{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}] {translated_text}")

        deepl_usage = deepl_translator.get_usage()

        if deepl_usage.character.valid:
          print(f"\nDeepL: Character usage: {deepl_usage.character.count} / {deepl_usage.character.limit} ({100*(deepl_usage.character.count/deepl_usage.character.limit):.2f}%)\n")
      elif source_language_code == target_language_code:
        print(f"Nothing to translate. Results are already in {result['language']}.")
      elif task == 'transcribe' and source_language_code not in deepl_source_languages:
        print(f"DeepL: {result['language']} is not yet supported")
  except deepl.DeepLException as e:
    if isinstance(e, deepl.AuthorizationException) and str(e) == "Authorization failure, check auth_key":
      e = "Authorization failure, check deepl_api_key"
    print(f"\nDeepL: [Error] {e}\n")

  # save translated results (if any)

  if translated_results:
    print("Writing translated results...")

    for audio_path, translated_result in translated_results.items():
      print(end='\n')

      translated_result['text'] = '\n'.join(map(lambda translated_segment: translated_segment['text'], translated_result['segments']))

      output_file_name = os.path.splitext(os.path.basename(audio_path))[0]
      translated_output_file_name = f"{output_file_name}_{deepl_target_language}"

      for output_format in output_formats:
        write_result(translated_result, output_format, translated_output_file_name)